# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
# import libraries
import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
import statsmodels.api as sm

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [6]:
# load Tables
train_flights_df = pd.read_csv('data/train_flights_df_random.csv', sep = ',')
train_flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-11-13,AA,AA_CODESHARE,AA,3721,MQ,N694AE,3721,11953,GNV,...,294,11.0,0.0,0.0,0.0,16.0,NaN,NaN,NaN,NaN
1,2019-07-20,UA,UA_CODESHARE,UA,6068,YV,N507MJ,6068,10792,BUF,...,283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-02-01,NK,NK,NK,731,NK,N682NK,731,13930,ORD,...,1744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-30,WN,WN,WN,4570,WN,N8311Q,4570,11697,FLL,...,1046,12.0,0.0,0.0,0.0,10.0,NaN,NaN,NaN,NaN
4,2019-01-19,AA,AA,AA,156,AA,N159AN,156,11697,FLL,...,1182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# check shape (# of rowns and columns)
train_flights_df.shape

(70000, 42)

In [8]:
# check data types
train_flights_df.dtypes

fl_date                 object
mkt_unique_carrier      object
branded_code_share      object
mkt_carrier             object
mkt_carrier_fl_num       int64
op_unique_carrier       object
tail_num                object
op_carrier_fl_num        int64
origin_airport_id        int64
origin                  object
origin_city_name        object
dest_airport_id          int64
dest                    object
dest_city_name          object
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
wheels_off             float64
wheels_on              float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
cancelled                int64
cancellation_code       object
diverted                 int64
dup                     object
crs_elapsed_time         int64
actual_elapsed_time    float64
air_time               float64
flights                  int64
distance

In [9]:
train_flights_df.isnull().sum()

fl_date                    0
mkt_unique_carrier         0
branded_code_share         0
mkt_carrier                0
mkt_carrier_fl_num         0
op_unique_carrier          0
tail_num                 206
op_carrier_fl_num          0
origin_airport_id          0
origin                     0
origin_city_name           0
dest_airport_id            0
dest                       0
dest_city_name             0
crs_dep_time               0
dep_time                1143
dep_delay               1158
taxi_out                1215
wheels_off              1215
wheels_on               1245
taxi_in                 1245
crs_arr_time               0
arr_time                1217
arr_delay               1383
cancelled                  0
cancellation_code      68804
diverted                   0
dup                        0
crs_elapsed_time           0
actual_elapsed_time     1368
air_time                1394
flights                    0
distance                   0
carrier_delay          56865
weather_delay 

In [10]:
# replace NaN in delay columns with mean and save ot original dataframe
train_flights_df.fillna(train_flights_df.mean(), inplace=True)
train_flights_df

C:\Users\Jagvir Dhesi\AppData\Local\Temp\ipykernel_22480\2632120511.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_flights_df.fillna(train_flights_df.mean(), inplace=True)


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-11-13,AA,AA_CODESHARE,AA,3721,MQ,N694AE,3721,11953,GNV,...,294,11.000000,0.000000,0.000000,0.000000,16.000000,1313.008715,40.015251,39.122004,NaN
1,2019-07-20,UA,UA_CODESHARE,UA,6068,YV,N507MJ,6068,10792,BUF,...,283,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN
2,2019-02-01,NK,NK,NK,731,NK,N682NK,731,13930,ORD,...,1744,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN
3,2019-11-30,WN,WN,WN,4570,WN,N8311Q,4570,11697,FLL,...,1046,12.000000,0.000000,0.000000,0.000000,10.000000,1313.008715,40.015251,39.122004,NaN
4,2019-01-19,AA,AA,AA,156,AA,N159AN,156,11697,FLL,...,1182,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,2018-12-30,AA,AA_CODESHARE,AA,2954,OO,N754SK,2954,15376,TUS,...,451,0.000000,0.000000,26.000000,0.000000,1.000000,1313.008715,40.015251,39.122004,NaN
69996,2019-06-24,B6,B6,B6,1266,B6,N955JB,1266,14635,RSW,...,1249,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN
69997,2019-06-11,DL,DL_CODESHARE,DL,5309,9E,N834AY,5309,10208,AGS,...,143,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN
69998,2018-02-23,AA,AA,AA,867,AA,N703UW,867,14100,PHL,...,1496,20.191778,3.450324,15.816368,0.157214,26.806928,1313.008715,40.015251,39.122004,NaN


In [14]:
# drop missing rows for tail_num
train_flights_df.dropna(subset=['tail_num'], inplace=True)

In [23]:
train_flights_df.isnull().sum()

fl_date                0
mkt_unique_carrier     0
mkt_carrier_fl_num     0
op_unique_carrier      0
tail_num               0
op_carrier_fl_num      0
origin_airport_id      0
origin                 0
origin_city_name       0
dest_airport_id        0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
diverted               0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [22]:
# dropping columns 
train_flights_df.drop(columns=['branded_code_share',
           'mkt_carrier', 'cancelled', 'cancellation_code', 'flights', 'air_time',
            'first_dep_time', 'total_add_gtime', 'longest_add_gtime', 'no_name'], inplace=True)

KeyError: "['branded_code_share', 'mkt_carrier', 'cancelled', 'cancellation_code', 'flights', 'air_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime', 'no_name'] not found in axis"

In [24]:
train_flights_df.isnull().sum()

fl_date                0
mkt_unique_carrier     0
mkt_carrier_fl_num     0
op_unique_carrier      0
tail_num               0
op_carrier_fl_num      0
origin_airport_id      0
origin                 0
origin_city_name       0
dest_airport_id        0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
diverted               0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [25]:
train_flights_df

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,...,diverted,dup,crs_elapsed_time,actual_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-11-13,AA,3721,MQ,N694AE,3721,11953,GNV,"Gainesville, FL",13303,...,0,N,69,68.000000,294,11.000000,0.000000,0.000000,0.000000,16.000000
1,2019-07-20,UA,6068,YV,N507MJ,6068,10792,BUF,"Buffalo, NY",12264,...,0,N,64,62.000000,283,20.191778,3.450324,15.816368,0.157214,26.806928
2,2019-02-01,NK,731,NK,N682NK,731,13930,ORD,"Chicago, IL",12892,...,0,N,282,247.000000,1744,20.191778,3.450324,15.816368,0.157214,26.806928
3,2019-11-30,WN,4570,WN,N8311Q,4570,11697,FLL,"Fort Lauderdale, FL",14843,...,0,N,155,146.000000,1046,12.000000,0.000000,0.000000,0.000000,10.000000
4,2019-01-19,AA,156,AA,N159AN,156,11697,FLL,"Fort Lauderdale, FL",13930,...,0,N,203,132.970131,1182,20.191778,3.450324,15.816368,0.157214,26.806928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,2018-12-30,AA,2954,OO,N754SK,2954,15376,TUS,"Tucson, AZ",12892,...,0,N,118,144.000000,451,0.000000,0.000000,26.000000,0.000000,1.000000
69996,2019-06-24,B6,1266,B6,N955JB,1266,14635,RSW,"Fort Myers, FL",10721,...,0,N,188,183.000000,1249,20.191778,3.450324,15.816368,0.157214,26.806928
69997,2019-06-11,DL,5309,9E,N834AY,5309,10208,AGS,"Augusta, GA",10397,...,0,N,73,54.000000,143,20.191778,3.450324,15.816368,0.157214,26.806928
69998,2018-02-23,AA,867,AA,N703UW,867,14100,PHL,"Philadelphia, PA",14683,...,0,N,265,240.000000,1496,20.191778,3.450324,15.816368,0.157214,26.806928


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [27]:
# select delay columns
train_flights_delays = train_flights_df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']]

In [29]:
# calculate stats for delays 
delay_stats =  train_flights_df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].describe()
delay_stats

,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,69794.000000,69794.000000,69794.000000,69794.000000,69794.000000
mean,20.191778,3.450324,15.816368,0.157214,26.806928
std,26.464093,12.356909,15.463673,3.674618,21.907628
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.191778,3.450324,15.816368,0.157214,26.806928
50%,20.191778,3.450324,15.816368,0.157214,26.806928
75%,20.191778,3.450324,15.816368,0.157214,26.806928
max,1445.000000,980.000000,1226.000000,927.000000,949.000000


In [31]:
# show above stats grouped by 'arr_delay'
delay_stats =  train_flights_df.groupby('arr_delay')[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].describe()
delay_stats

carrier_delay                                             \
                  count         mean std          min          25%   
arr_delay                                                            
-100.0              1.0    20.191778 NaN    20.191778    20.191778   
-85.0               1.0    20.191778 NaN    20.191778    20.191778   
-75.0               1.0    20.191778 NaN    20.191778    20.191778   
-72.0               1.0    20.191778 NaN    20.191778    20.191778   
-70.0               1.0    20.191778 NaN    20.191778    20.191778   
...                 ...          ...  ..          ...          ...   
 1226.0             1.0     0.000000 NaN     0.000000     0.000000   
 1315.0             1.0  1067.000000 NaN  1067.000000  1067.000000   
 1318.0             1.0  1318.000000 NaN  1318.000000  1318.000000   
 1423.0             1.0  1394.000000 NaN  1394.000000  1394.000000   
 1445.0             1.0  1445.000000 NaN  1445.000000  1445.000000   

                                                 weather_delay            ...  \
                   50%          75%          max         count      mean  ...   
arr_delay                                                                 ...   
-100.0       20.191778    20.191778    20.191778           1.0  3.450324  ...   
-85.0        20.191778    20.191778    20.191778           1.0  3.450324  ...   
-75.0        20.191778    20.191778    20.191778           1.0  3.450324  ...   
-72.0        20.191778    20.191778    20.191778           1.0  3.450324  ...   
-70.0        20.191778    20.191778    20.191778           1.0  3.450324  ...   
...                ...          ...          ...           ...       ...  ...   
 1226.0       0.000000     0.000000     0.000000           1.0  0.000000  ...   
 1315.0    1067.000000  1067.000000  1067.000000           1.0  0.000000  ...   
 1318.0    1318.000000  1318.000000  1318.000000           1.0  0.000000  ...   
 1423.0    1394.000000  1394.000000  1394.000000           1.0  0.000000  ...   
 1445.0    1445.000000  1445.000000  1445.000000           1.0  0.000000  ...   

          security_delay           late_aircraft_delay                  \
                     75%       max               count        mean std   
arr_delay                                                                
-100.0          0.157214  0.157214                 1.0   26.806928 NaN   
-85.0           0.157214  0.157214                 1.0   26.806928 NaN   
-75.0           0.157214  0.157214                 1.0   26.806928 NaN   
-72.0           0.157214  0.157214                 1.0   26.806928 NaN   
-70.0           0.157214  0.157214                 1.0   26.806928 NaN   
...                  ...       ...                 ...         ...  ..   
 1226.0         0.000000  0.000000                 1.0    0.000000 NaN   
 1315.0         0.000000  0.000000                 1.0  248.000000 NaN   
 1318.0         0.000000  0.000000                 1.0    0.000000 NaN   
 1423.0         0.000000  0.000000                 1.0    0.000000 NaN   
 1445.0         0.000000  0.000000                 1.0    0.000000 NaN   

                                                                       
                  min         25%         50%         75%         max  
arr_delay                                                              
-100.0      26.806928   26.806928   26.806928   26.806928   26.806928  
-85.0       26.806928   26.806928   26.806928   26.806928   26.806928  
-75.0       26.806928   26.806928   26.806928   26.806928   26.806928  
-72.0       26.806928   26.806928   26.806928   26.806928   26.806928  
-70.0       26.806928   26.806928   26.806928   26.806928   26.806928  
...               ...         ...         ...         ...         ...  
 1226.0      0.000000    0.000000    0.000000    0.000000    0.000000  
 1315.0    248.000000  248.000000  248.000000  248.000000  248.000000  
 1318.0      0.000000    0.000000    0.000000    0.000000    0.00

In [32]:
# taking apart 'fl_date' column to make year, month & date columns
train_flights_df['fl_date'] = pd.to_datetime(train_flights_df['fl_date'], errors='coerce')
train_flights_df['year'] = train_flights_df['fl_date'].dt.year
train_flights_df['month'] = train_flights_df['fl_date'].dt.month
train_flights_df['day'] = train_flights_df['fl_date'].dt.day
train_flights_df.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,...,actual_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,year,month,day
0,2018-11-13,AA,3721,MQ,N694AE,3721,11953,GNV,"Gainesville, FL",13303,...,68.000000,294,11.000000,0.000000,0.000000,0.000000,16.000000,2018,11,13
1,2019-07-20,UA,6068,YV,N507MJ,6068,10792,BUF,"Buffalo, NY",12264,...,62.000000,283,20.191778,3.450324,15.816368,0.157214,26.806928,2019,7,20
2,2019-02-01,NK,731,NK,N682NK,731,13930,ORD,"Chicago, IL",12892,...,247.000000,1744,20.191778,3.450324,15.816368,0.157214,26.806928,2019,2,1
3,2019-11-30,WN,4570,WN,N8311Q,4570,11697,FLL,"Fort Lauderdale, FL",14843,...,146.000000,1046,12.000000,0.000000,0.000000,0.000000,10.000000,2019,11,30
4,2019-01-19,AA,156,AA,N159AN,156,11697,FLL,"Fort Lauderdale, FL",13930,...,132.970131,1182,20.191778,3.450324,15.816368,0.157214,26.806928,2019,1,19


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.